# Part 1: 开发测试 - 数据获取 (Tushare & yfinance)

这个Notebook用于测试我们项目的基础模块是否正常工作。

**核心测试目标**:
1. `config_manager`: 能否正确从 `.env` 文件加载 Tushare Token 和其他API密钥。
2. `data_fetcher`: 能否根据股票代码自动选择 `yfinance` 或 `Tushare`，并成功获取数据。

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd

# 将上级目录（项目根目录）添加到系统路径
# 这样我们就可以直接 'from src import ...'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"项目根目录 '{project_root}' 已添加到系统路径。")

In [ ]:
from src.config_manager import get_tushare_token, get_llm_api_key

print("--- 测试配置管理器 ---")

# 测试Tushare Token
tushare_token = get_tushare_token()
if tushare_token and len(tushare_token) > 10:
    print("✅ Tushare Token: 成功加载。")
else:
    print("❌ Tushare Token: 未加载或格式不正确。请检查 .env 文件中的 TUSHARE_TOKEN。")

# 测试一个LLM API Key (以Gemini为例)
gemini_key = get_llm_api_key('gemini')
if gemini_key:
     print("✅ Gemini API Key: 成功加载。")
else:
     print("⚠️ Gemini API Key: 未在 .env 文件中配置 (这是可选的，在此阶段不影响数据获取)。")

In [ ]:
from src.data_fetcher import fetch_stock_data

### 测试 1: 获取美股数据 (使用 yfinance)

我们将尝试获取苹果公司 (AAPL) 的数据。这应该会触发 `data_fetcher` 中的 `yfinance` 逻辑。

In [ ]:
aapl_df = fetch_stock_data(ticker='AAPL', period='1y')

if aapl_df is not None and not aapl_df.empty:
    print("\n✅ 成功获取苹果公司(AAPL)数据！")
    print("数据维度:", aapl_df.shape)
    print("\n数据预览 (前5行):")
    display(aapl_df.head())
    print("\n数据预览 (后5行):")
    display(aapl_df.tail())
else:
    print("\n❌ 获取AAPL数据失败。")

### 测试 2: 获取中国A股数据 (使用 Tushare)

我们将尝试获取贵州茅台 (600519) 的数据。这应该会触发 `data_fetcher` 中的 `Tushare` 逻辑。
**注意**: 此单元格的成功运行依赖于你在 `.env` 文件中正确配置了 `TUSHARE_TOKEN`。

In [ ]:
# 使用A股代码（6位数字）
moutai_df = fetch_stock_data(ticker='600519', period='1y')

if moutai_df is not None and not moutai_df.empty:
    print("\n✅ 成功获取贵州茅台(600519)数据！")
    print("数据维度:", moutai_df.shape)
    print("\n数据预览 (前5行):")
    display(moutai_df.head())
    print("\n数据预览 (后5行):")
    display(moutai_df.tail())
else:
    print("\n❌ 获取600519数据失败。请检查Tushare Token是否配置正确以及网络连接。")

In [ ]:
# 测试深圳市场的股票
pingan_df = fetch_stock_data(ticker='000001.SZ', period='6mo')

if pingan_df is not None and not pingan_df.empty:
    print("\n✅ 成功获取平安银行(000001.SZ)数据！")
    print("数据维度:", pingan_df.shape)
    print("\n数据预览 (后5行):")
    display(pingan_df.tail())
else:
    print("\n❌ 获取000001.SZ数据失败。")

### 测试 3: 错误处理

我们将尝试获取一个不存在的股票代码，以验证我们的函数能否优雅地处理错误并返回 `None`。

In [ ]:
invalid_df = fetch_stock_data(ticker='THIS_IS_A_FAKE_TICKER_XYZ')

if invalid_df is None:
    print("\n✅ 成功处理了无效代码，函数按预期返回了 None。")
else:
    print("\n❌ 错误处理测试失败，函数没有返回 None。")